In [0]:
# some libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving nga_series.csv to nga_series.csv


In [3]:
# to read the dataframe
import io
df = pd.read_csv(io.StringIO(uploaded['nga_series.csv'].decode('utf-8')))
df

,Date,NGA_TM_TAX_MANF_BR_ZS,NGA_TM_TAX_MRCH_BR_ZS,NGA_FM_LBL_BMNY_ZG,NGA_FM_LBL_BMNY_GD_ZS,NGA_FM_LBL_BMNY_CN,NGA_FM_LBL_BMNY_IR_ZS,NGA_GC_BAL_CASH_GD_ZS,NGA_IC_BUS_DISC_XQ,NGA_IQ_WEF_CUST_XQ,NGA_GC_BAL_CASH_CN,NGA_GC_DOD_TOTL_GD_ZS,NGA_AG_YLD_CREL_KG,NGA_AG_PRD_CREL_MT,NGA_GC_DOD_TOTL_CN,NGA_NE_GDI_STKB_CN,NGA_NE_GDI_STKB_KN,NGA_BN_RES_INCL_CD,NGA_SL_AGR_0714_ZS,NGA_NE_GDI_STKB_CD,NGA_NV_MNF_CHEM_ZS_UN,NGA_SL_AGR_0714_MA_ZS,NGA_SL_MNF_0714_ZS,NGA_SL_AGR_0714_FE_ZS,NGA_SL_MNF_0714_FE_ZS,NGA_SL_SRV_0714_ZS,NGA_SL_SRV_0714_FE_ZS,NGA_SL_MNF_0714_MA_ZS,NGA_SL_SRV_0714_MA_ZS,NGA_SE_PRM_UNER_FE,NGA_SE_PRM_UNER,NGA_SH_HIV_0014,NGA_SE_PRM_UNER_MA,NGA_FM_AST_CGOV_ZG_M3,NGA_SH_MLR_TRET_ZS,NGA_FS_AST_DOMO_GD_ZS,NGA_FS_AST_CGOV_GD_ZS,NGA_FM_AST_PRVT_ZG_M3,NGA_FM_AST_DOMO_ZG_M3,NGA_EN_ATM_CO2E_KD_GD,...,NGA_NV_AGR_TOTL_KD_ZG,NGA_NV_AGR_TOTL_ZS,NGA_NV_AGR_TOTL_CN,NGA_NV_AGR_TOTL_KN,NGA_IS_AIR_PSGR,NGA_NV_AGR_TOTL_CD,NGA_IS_AIR_GOOD_MT_K1,NGA_ER_H2O_FWAG_ZS,NGA_EG_USE_COMM_CL_ZS,NGA_IS_AIR_DPRT,NGA_ER_H2O_FWIN_ZS,NGA_ER_H2O_FWDM_ZS,NGA_ER_H2O_FWTL_ZS,NGA_ER_H2O_FWTL_K3,NGA_AG_LND_ARBL_ZS,NGA_SH_HIV_ARTC_ZS,NGA_AG_LND_ARBL_HA,NGA_AG_LND_ARBL_HA_PC,NGA_SH_STA_ARIC_ZS,NGA_MS_MIL_TOTL_TF_ZS,NGA_MS_MIL_TOTL_P1,NGA_FB_ATM_TOTL_P5,NGA_MS_MIL_MPRT_KD,NGA_MS_MIL_XPRT_KD,NGA_AG_LND_PRCP_MM,NGA_IC_CUS_DURS_EX,NGA_IC_TAX_METG,NGA_FB_BNK_CAPA_ZS,NGA_FD_RES_LIQU_AS_ZS,NGA_FB_AST_NPER_ZS,NGA_VC_BTL_DETH,NGA_TM_TAX_MRCH_BC_ZS,NGA_TM_TAX_MANF_BC_ZS,NGA_TM_TAX_TCOM_BC_ZS,NGA_SP_DYN_CBRT_IN,NGA_EN_BIR_THRD_NO,NGA_FB_CBK_BRWR_P3,NGA_SH_STA_BRTC_ZS,NGA_TM_TAX_TCOM_BR_ZS,CPI
0,1995-12-31,68.44,115.29,19.411715,10.281913,3.187635e+11,2.650768,NaN,NaN,NaN,NaN,NaN,1210.5,22512600.0,NaN,1.045843e+09,8.940942e+09,-8.626524e+08,NaN,1.486417e+07,25.465605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63000.0,NaN,-9.406721,NaN,NaN,8.483317,22.335131,NaN,0.246183,...,3.586410,25.486506,7.901416e+11,3.977382e+12,5.479000e+05,1.122998e+10,1.900000,NaN,0.381946,6600.000000,NaN,NaN,NaN,NaN,36.123280,NaN,32900000.0,0.304597,NaN,0.266649,89100.0,NaN,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.53,11.25,56.58,43.443,NaN,NaN,NaN,146.76,6.9
1,1996-12-31,67.44,115.01,16.178159,9.063329,3.703335e+11,1.069242,NaN,NaN,NaN,NaN,NaN,1220.4,21665000.0,NaN,1.793278e+09,1.227920e+10,-2.384873e+08,NaN,2.241597e+07,25.706182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000.0,NaN,-47.852793,NaN,NaN,2.703470,15.844491,NaN,0.261482,...,3.926359,26.199159,1.070515e+12,4.133548e+12,2.212000e+05,1.338143e+10,4.700000,NaN,0.343117,5600.000000,NaN,NaN,NaN,NaN,36.342875,NaN,33100000.0,0.298917,NaN,0.260236,89100.0,NaN,15000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,21.48,11.15,57.28,43.363,NaN,NaN,NaN,146.82,6.9
2,1997-12-31,67.44,115.01,16.039003,9.725269,4.297313e+11,0.680632,NaN,NaN,NaN,NaN,NaN,1182.8,21853000.0,NaN,2.439006e+09,1.540889e+10,4.670708e+06,NaN,3.005924e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88000.0,NaN,-17.310651,NaN,NaN,1.049139,15.761332,NaN,0.271070,...,4.164253,27.416651,1.211462e+12,4.305680e+12,3.182000e+05,1.493051e+10,5.000000,NaN,0.332273,6400.000000,NaN,NaN,NaN,NaN,36.452672,NaN,33200000.0,0.292453,NaN,0.304652,107000.0,NaN,1000000.0,NaN,1150.0,NaN,NaN,NaN,NaN,NaN,NaN,21.48,11.15,57.28,43.303,NaN,NaN,NaN,146.82,17.6
3,1998-12-31,67.44,115.01,22.317758,10.939035,5.256377e+11,0.818404,NaN,NaN,NaN,NaN,NaN,1201.8,22040000.0,NaN,2.957541e+09,1.819606e+10,-8.184873e+08,NaN,3.360842e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000.0,NaN,21.771239,NaN,NaN,2.911794,12.906972,NaN,0.236412,...,3.938095,27.908371,1.341041e+12,4.475241e+12,3.134000e+05,1.523910e+10,27.000000,NaN,0.350200,7500.000000,NaN,NaN,NaN,NaN,37.331050,NaN,34000000.0,0.292132,NaN,0.296818,107000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.400000,NaN,21.48,11.15,57.28,43.254,NaN,NaN,NaN,146.82,19.0
4,1999-12-31,67.44,115.01,33.121058,12.763394,6.997345e+11,1.341297,NaN,NaN,NaN,NaN,NaN,1239.3,22405000.0,NaN,3.498310e+09,2.099023e+10,-3.537663e+09,NaN,3.788588e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3990429.0,7029016.0,11

In [0]:
import datetime
def datetiem(x):
    date_time_obj = datetime.datetime.strptime(x, '%Y-%m-%d' )
    return date_time_obj

In [0]:
df["Date"] = df["Date"].apply(lambda x: datetiem(x))
df.index = df.Date # index as date because time is not a variable
df=df.drop(['Date'], axis=1)

In [25]:
# df0 = df.fillna(0)
# removes columns where more than 30% of data is missing
# rationale for 0.7, if we can train test split for 0.7/ 0.3 we can accept at least 70% filled data
# fills remaining missing values with the mean of the column   
df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.85].fillna(df.mean())
# df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.interpolate())
df_mean

,NGA_TM_TAX_MANF_BR_ZS,NGA_TM_TAX_MRCH_BR_ZS,NGA_FM_LBL_BMNY_ZG,NGA_FM_LBL_BMNY_GD_ZS,NGA_FM_LBL_BMNY_CN,NGA_FM_LBL_BMNY_IR_ZS,NGA_AG_YLD_CREL_KG,NGA_AG_PRD_CREL_MT,NGA_NE_GDI_STKB_CN,NGA_NE_GDI_STKB_KN,NGA_BN_RES_INCL_CD,NGA_NE_GDI_STKB_CD,NGA_SH_HIV_0014,NGA_FM_AST_CGOV_ZG_M3,NGA_FS_AST_CGOV_GD_ZS,NGA_FM_AST_PRVT_ZG_M3,NGA_EN_ATM_CO2E_KD_GD,NGA_EN_ATM_CO2E_PP_GD_KD,NGA_EN_ATM_CO2E_PC,NGA_EN_ATM_CO2E_KT,NGA_EN_ATM_CO2E_PP_GD,NGA_EN_CO2_ETOT_ZS,NGA_EN_ATM_CO2E_LF_ZS,NGA_EN_ATM_CO2E_GF_KT,NGA_EN_ATM_CO2E_GF_ZS,NGA_EN_CO2_MANF_ZS,NGA_EN_ATM_CO2E_LF_KT,NGA_EN_CO2_OTHX_ZS,NGA_EN_ATM_CO2E_SF_ZS,NGA_EN_CO2_BLDG_ZS,NGA_EN_CO2_TRAN_ZS,NGA_EN_ATM_CO2E_SF_KT,NGA_EG_USE_CRNW_ZS,NGA_NY_GDP_COAL_RT_ZS,NGA_EN_ATM_CO2E_EG_ZS,NGA_DT_NFL_PCBO_CD,NGA_TM_VAL_SERV_CD_WT,NGA_TX_VAL_SERV_CD_WT,NGA_BX_GSR_CMCP_ZS,NGA_BM_GSR_CMCP_ZS,...,NGA_NY_ADJ_DNGY_CD,NGA_NY_ADJ_DNGY_GN_ZS,NGA_NY_ADJ_ICTR_GN_ZS,NGA_NY_ADJ_DMIN_GN_ZS,NGA_NY_ADJ_DRES_GN_ZS,NGA_NY_ADJ_DMIN_CD,NGA_NY_ADJ_DFOR_CD,NGA_NY_ADJ_DFOR_GN_ZS,NGA_NY_ADJ_NNAT_GN_ZS,NGA_NY_ADJ_NNAT_CD,NGA_NY_ADJ_DPEM_GN_ZS,NGA_NY_ADJ_DPEM_CD,NGA_SP_ADO_TFRT,NGA_SP_POP_DPND,NGA_SP_POP_DPND_YG,NGA_SP_POP_DPND_OL,NGA_AG_LND_AGRI_ZS,NGA_AG_LND_AGRI_K2,NGA_EA_PRD_AGRI_KD,NGA_NV_AGR_TOTL_KD,NGA_NV_AGR_TOTL_KD_ZG,NGA_NV_AGR_TOTL_ZS,NGA_NV_AGR_TOTL_CN,NGA_NV_AGR_TOTL_KN,NGA_IS_AIR_PSGR,NGA_NV_AGR_TOTL_CD,NGA_IS_AIR_GOOD_MT_K1,NGA_EG_USE_COMM_CL_ZS,NGA_IS_AIR_DPRT,NGA_AG_LND_ARBL_ZS,NGA_AG_LND_ARBL_HA,NGA_AG_LND_ARBL_HA_PC,NGA_MS_MIL_TOTL_TF_ZS,NGA_MS_MIL_TOTL_P1,NGA_TM_TAX_MRCH_BC_ZS,NGA_TM_TAX_MANF_BC_ZS,NGA_TM_TAX_TCOM_BC_ZS,NGA_SP_DYN_CBRT_IN,NGA_TM_TAX_TCOM_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995-12-31,68.44,115.29,19.411715,10.281913,3.187635e+11,2.650768,1210.5,22512600.0,1.045843e+09,8.940942e+09,-8.626524e+08,1.486417e+07,63000.0,-9.406721,8.483317,22.335131,0.246183,0.110976,0.331828,35841.258,0.152246,33.140067,68.129732,10073.249000,28.105177,9.612450,24418.553000,0.061031,0.143237,11.687519,45.498932,51.338000,79.510042,0.000846,0.486359,-4.484010e+08,1.368691e+09,1.890624e+08,80.241398,56.580704,...,4.054225e+09,9.689398,43.752390,0.000217,12.337607,9.086929e+04,1.107969e+09,2.647992,38.391538,1.606374e+10,2.002606,8.379277e+08,139.3548,89.076132,83.645360,5.430772,75.101288,684000.0,2090.146627,2.646331e+10,3.586410,25.486506,7.901416e+11,3.977382e+12,5.479000e+05,1.122998e+10,1.900000,0.381946,6600.000000,36.123280,32900000.0,0.304597,0.266649,89100.0,21.53,11.25,56.58,43.443,146.76,6.9
1996-12-31,67.44,115.01,16.178159,9.063329,3.703335e+11,1.069242,1220.4,21665000.0,1.793278e+09,1.227920e+10,-2.384873e+08,2.241597e+07,75000.0,-47.852793,2.703470,15.844491,0.261482,0.117873,0.358213,39665.939,0.158808,31.859379,71.008598,10208.928000,25.737265,9.667674,28166.227000,0.054930,0.055468,11.370503,47.047514,22.002000,78.307819,0.000254,0.516322,-6.902220e+08,1.422761e+09,2.295520e+08,84.481141,62.608043,...,5.500615e+09,11.261377,40.393385,0.000236,13.615732,1.151949e+05,1.149869e+09,2.354119,35.819708,1.749612e+10,1.939220,9.472115e+08,137.9954,88.819885,83.389031,5.430853,75.320882,686000.0,2181.961980,2.750235e+10,3.926359,26.199159,1.070515e+12,4.133548e+12,2.212000e+05,1.338143e+10,4.700000,0.343117,5600.000000,36.342875,33100000.0,0.298917,0.260236,89100.0,21.48,11.15,57.28,43.363,146.82,6.9
1997-12-31,67.44,115.01,16.039003,9.725269,4.297313e+11,0.680632,1182.8,21853000.0,2.439006e+09,1.540889e+10,4.670708e+06,3.005924e+07,88000.0,-17.310651,1.049139,15.761332,0.271070,0.122195,0.372861,42328.181,0.161861,29.224075,71.740449,10674.637000,25.218747,7.154943,30366.427000,0.048344,0.060643,10.635726,52.961083,25.669000,77.773430,0.000238,0.532924,-2.578740e+08,1.431956e+09,2.398900e+08,81.044871,43.331174,...,4.620927e+09,8.848677,43.050943,0.000051,11.044693,2.651577e+04,1.146770e+09,2.195966,38.936114,2.033309e+10,1.971375,1.029485e+09,136.6360,88.378216,82.970381,5.407834,75.540477,688000.0,2281.345133,2.864762e+10,4.164253,27.416651,1.21146

In [0]:
CPI_list = list(df_mean.CPI)
def pct_df(df):
    df.drop(["CPI"], axis=1)
    new_df = df.pct_change()
    new_df.replace(np.nan, 0)
    new_df = new_df.replace([np.inf, -np.inf], 0)
    new_df = new_df.dropna(how='all')
    new_df['CPI']=CPI_list[1:]
    new_df = new_df.fillna(df.mean())
    return new_df
def UpsampleByTimeIndex(df,bytime,method):
    upsampled = df.resample(bytime)
    interpolated = upsampled.interpolate(method=method)
    return interpolated
# Ignore the index and treat the values as equally space
# ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’
# .agg aggregates/ groups by specified time frame
# D for days, M for months
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.aggregate.html 
# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

In [0]:
pct_df_mean = pct_df(df_mean)

In [10]:
#Ignore this code; helper functions for binarizing
# list_col_names = list(pct_df_mean.columns.values)
from itertools import repeat
def binarize(x):
    if x< 0:
        x = -1
    elif x>0:
        x=1
    else:
        x = 0
    return x
month_cpi = [x for item in CPI_list[1:len(CPI_list)-1] for x in repeat(item, 12)]
month_cpi.insert(len(a),CPI_list[len(CPI_list)-1])
def bin_action_cpi(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action(df):
    df = df.applymap(binarize)
    return df
def binarize10(x):
    if x<= 0:
        x = 0
    else:
        x = 1
    return x
def bin_action_cpi10(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize10)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action10(df):
    df = df.applymap(binarize10)
    return df
def binarize123(x):
    if x< 0:
        x = 1
    elif x>0:
        x=3
    else:
        x = 2
    return x
def bin_action_cpi123(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize123)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action123(df):
    df = df.applymap(binarize123)
    return df

NameError: ignored

In [0]:
# percentage change df and binarize 123
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi123(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action123(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize 0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
# df_latest = bin_action_cpi10(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
# df_latest_wo_cpi = bin_action10(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize -1,0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df
df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
df_latest.index = list(range(len(df_latest)))
df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# mean df
# df_mean_wo_cpi = df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(df_mean,'Y','linear')
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

In [29]:
df_latest_wo_cpi

,NGA_TM_TAX_MANF_BR_ZS,NGA_TM_TAX_MRCH_BR_ZS,NGA_FM_LBL_BMNY_ZG,NGA_FM_LBL_BMNY_GD_ZS,NGA_FM_LBL_BMNY_CN,NGA_FM_LBL_BMNY_IR_ZS,NGA_AG_YLD_CREL_KG,NGA_AG_PRD_CREL_MT,NGA_NE_GDI_STKB_CN,NGA_NE_GDI_STKB_KN,NGA_BN_RES_INCL_CD,NGA_NE_GDI_STKB_CD,NGA_SH_HIV_0014,NGA_FM_AST_CGOV_ZG_M3,NGA_FS_AST_CGOV_GD_ZS,NGA_FM_AST_PRVT_ZG_M3,NGA_EN_ATM_CO2E_KD_GD,NGA_EN_ATM_CO2E_PP_GD_KD,NGA_EN_ATM_CO2E_PC,NGA_EN_ATM_CO2E_KT,NGA_EN_ATM_CO2E_PP_GD,NGA_EN_CO2_ETOT_ZS,NGA_EN_ATM_CO2E_LF_ZS,NGA_EN_ATM_CO2E_GF_KT,NGA_EN_ATM_CO2E_GF_ZS,NGA_EN_CO2_MANF_ZS,NGA_EN_ATM_CO2E_LF_KT,NGA_EN_CO2_OTHX_ZS,NGA_EN_ATM_CO2E_SF_ZS,NGA_EN_CO2_BLDG_ZS,NGA_EN_CO2_TRAN_ZS,NGA_EN_ATM_CO2E_SF_KT,NGA_EG_USE_CRNW_ZS,NGA_NY_GDP_COAL_RT_ZS,NGA_EN_ATM_CO2E_EG_ZS,NGA_DT_NFL_PCBO_CD,NGA_TM_VAL_SERV_CD_WT,NGA_TX_VAL_SERV_CD_WT,NGA_BX_GSR_CMCP_ZS,NGA_BM_GSR_CMCP_ZS,...,NGA_NY_ADJ_AEDU_CD,NGA_NY_ADJ_DNGY_CD,NGA_NY_ADJ_DNGY_GN_ZS,NGA_NY_ADJ_ICTR_GN_ZS,NGA_NY_ADJ_DMIN_GN_ZS,NGA_NY_ADJ_DRES_GN_ZS,NGA_NY_ADJ_DMIN_CD,NGA_NY_ADJ_DFOR_CD,NGA_NY_ADJ_DFOR_GN_ZS,NGA_NY_ADJ_NNAT_GN_ZS,NGA_NY_ADJ_NNAT_CD,NGA_NY_ADJ_DPEM_GN_ZS,NGA_NY_ADJ_DPEM_CD,NGA_SP_ADO_TFRT,NGA_SP_POP_DPND,NGA_SP_POP_DPND_YG,NGA_SP_POP_DPND_OL,NGA_AG_LND_AGRI_ZS,NGA_AG_LND_AGRI_K2,NGA_EA_PRD_AGRI_KD,NGA_NV_AGR_TOTL_KD,NGA_NV_AGR_TOTL_KD_ZG,NGA_NV_AGR_TOTL_ZS,NGA_NV_AGR_TOTL_CN,NGA_NV_AGR_TOTL_KN,NGA_IS_AIR_PSGR,NGA_NV_AGR_TOTL_CD,NGA_IS_AIR_GOOD_MT_K1,NGA_EG_USE_COMM_CL_ZS,NGA_IS_AIR_DPRT,NGA_AG_LND_ARBL_ZS,NGA_AG_LND_ARBL_HA,NGA_AG_LND_ARBL_HA_PC,NGA_MS_MIL_TOTL_TF_ZS,NGA_MS_MIL_TOTL_P1,NGA_TM_TAX_MRCH_BC_ZS,NGA_TM_TAX_MANF_BC_ZS,NGA_TM_TAX_TCOM_BC_ZS,NGA_SP_DYN_CBRT_IN,NGA_TM_TAX_TCOM_BR_ZS
0,-0.014611,-0.002429,-0.166578,-0.118517,0.161782,-0.596629,0.008178,-0.037650,0.714672,0.373368,-0.723542,0.508054,0.190476,4.087085,-0.681319,-0.290602,0.062145,0.062145,0.079512,0.106712,0.043102,-0.038645,0.042256,0.013469,-0.084252,0.005745,0.153476,-0.099973,-0.612752,-0.027124,0.034036,-0.571429,-0.015120,-0.699383,0.061605,0.539296,0.039504,0.214160,0.052837,0.106526,...,0.167370,0.356761,0.162237,-0.076773,0.085944,0.103596,0.267699,0.037816,-0.110979,-0.066990,0.089169,-0.031651,0.130421,-0.009755,-0.002877,-0.003064,0.000015,0.002924,0.002924,0.043928,0.039264,0.094788,0.027962,0.354839,0.039264,-0.596277,0.191581,1.473684,-0.101662,-0.151515,0.006079,0.006079,-0.018647,-0.024052,0.000000,-0.002322,-0.008889,0.012372,-0.001841,0.000409
1,-0.013394,-0.002226,-0.153413,-0.102555,0.161666,-0.577197,0.004929,-0.033789,0.685123,0.363494,-0.748212,0.494131,0.189048,3.693307,-0.675537,-0.266823,0.060022,0.060022,0.076294,0.103412,0.041113,-0.042317,0.039593,0.016148,-0.078910,-0.016393,0.147196,-0.101633,-0.553916,-0.030249,0.041674,-0.509921,-0.014429,-0.646435,0.059151,0.442156,0.036751,0.200066,0.045045,0.071991,...,0.159184,0.313704,0.130863,-0.064893,0.013390,0.079227,0.181240,0.034441,-0.107330,-0.054157,0.095251,-0.027632,0.126791,-0.009763,-0.003051,-0.003227,-0.000339,0.002923,0.002923,0.044063,0.039462,0.091938,0.029504,0.336241,0.039462,-0.510044,0.185263,1.356196,-0.095824,-0.126984,0.005824,0.005824,-0.018895,-0.007825,0.016741,-0.002129,-0.008148,0.011341,-0.001803,0.000375
2,-0.012176,-0.002024,-0.140248,-0.086592,0.161550,-0.557765,0.001680,-0.029929,0.655574,0.353619,-0.772882,0.480208,0.187619,3.299529,-0.669754,-0.243043,0.057899,0.057899,0.073076,0.100112,0.039123,-0.045990,0.036931,0.018827,-0.073568,-0.038531,0.140916,-0.103293,-0.495079,-0.033374,0.049312,-0.448413,-0.013738,-0.593487,0.056697,0.345015,0.033997,0.185973,0.037252,0.037456,...,0.150997,0.270647,0.099490,-0.053012,-0.059163,0.054858,0.094780,0.031065,-0.103680,-0.041324,0.101332,-0.023613,0.123161,-0.009771,-0.003226,-0.003390,-0.000694,0.002923,0.002923,0.044198,0.039660,0.089088,0.031047,0.317643,0.039660,-0.423811,0.178945,1.238708,-0.089986,-0.102453,0.005569,0.005569,-0.019144,0.008403,0.033483,-0.001935,-0.007407,0.010310,-0.001765,0.000341
3,-0.010959,-0.001821,-0.127084,-0.070629,0.161434,-0.538333,-0.001569,-0.026068,0.626025,0.343745,-0

In [30]:
# check for null, if false then no missing values
def null_check(dfr):
    return dfr.isnull().values.any()
# print(null_check(df0))
# print(null_check(df_mean))
print(null_check(df_latest_wo_cpi))

False


In [31]:
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))

input_data = df_latest.values
test_data = df_latest_wo_cpi.values
input_data[:,0:len(input_data[0])-1] = sc.fit_transform(input_data[:,0:len(input_data[0])-1])
test_data[:,0:len(test_data[0])] = sc.fit_transform(test_data[:,:])
test_data

array([[0.70415577, 0.        , 0.15185556, ..., 1.        , 0.95768026,
        1.        ],
       [0.71877637, 0.00727355, 0.15556226, ..., 0.97277166, 0.95988668,
        0.96664623],
       [0.73339696, 0.01454711, 0.15926895, ..., 0.94554332, 0.96209309,
        0.93329247],
       ...,
       [0.8996691 , 0.21943233, 0.85129588, ..., 0.56104994, 0.35776505,
        0.49979567],
       [0.88963601, 0.15335749, 0.92564794, ..., 0.61715493, 0.35477213,
        0.54977523],
       [0.87960292, 0.08728266, 1.        , ..., 0.67325992, 0.3517792 ,
        0.5997548 ]])

In [15]:
print(test_data.shape)
print(input_data.shape) # check data and shape, are there 22/ 229 rows?

(241, 609)
(241, 610)


In [16]:
df_latest.head()

,NGA_TM_TAX_MANF_BR_ZS,NGA_TM_TAX_MRCH_BR_ZS,NGA_FM_LBL_BMNY_ZG,NGA_FM_LBL_BMNY_GD_ZS,NGA_FM_LBL_BMNY_CN,NGA_FM_LBL_BMNY_IR_ZS,NGA_AG_YLD_CREL_KG,NGA_AG_PRD_CREL_MT,NGA_NE_GDI_STKB_CN,NGA_NE_GDI_STKB_KN,NGA_BN_RES_INCL_CD,NGA_NE_GDI_STKB_CD,NGA_SH_HIV_0014,NGA_FM_AST_CGOV_ZG_M3,NGA_FS_AST_CGOV_GD_ZS,NGA_FM_AST_PRVT_ZG_M3,NGA_EN_ATM_CO2E_KD_GD,NGA_EN_ATM_CO2E_PP_GD_KD,NGA_EN_ATM_CO2E_PC,NGA_EN_ATM_CO2E_KT,NGA_EN_ATM_CO2E_PP_GD,NGA_EN_CO2_ETOT_ZS,NGA_EN_ATM_CO2E_LF_ZS,NGA_EN_ATM_CO2E_GF_KT,NGA_EN_ATM_CO2E_GF_ZS,NGA_EN_CO2_MANF_ZS,NGA_EN_ATM_CO2E_LF_KT,NGA_EN_CO2_OTHX_ZS,NGA_EN_ATM_CO2E_SF_ZS,NGA_EN_CO2_BLDG_ZS,NGA_EN_CO2_TRAN_ZS,NGA_EN_ATM_CO2E_SF_KT,NGA_EG_USE_CRNW_ZS,NGA_NY_GDP_COAL_RT_ZS,NGA_EN_ATM_CO2E_EG_ZS,NGA_DT_NFL_PCBO_CD,NGA_TM_VAL_SERV_CD_WT,NGA_TX_VAL_SERV_CD_WT,NGA_BX_GSR_CMCP_ZS,NGA_BM_GSR_CMCP_ZS,...,NGA_NY_ADJ_DRES_GN_ZS,NGA_NY_ADJ_DMIN_CD,NGA_NY_ADJ_DFOR_CD,NGA_NY_ADJ_DFOR_GN_ZS,NGA_NY_ADJ_NNAT_GN_ZS,NGA_NY_ADJ_NNAT_CD,NGA_NY_ADJ_DPEM_GN_ZS,NGA_NY_ADJ_DPEM_CD,NGA_SP_ADO_TFRT,NGA_SP_POP_DPND,NGA_SP_POP_DPND_YG,NGA_SP_POP_DPND_OL,NGA_AG_LND_AGRI_ZS,NGA_AG_LND_AGRI_K2,NGA_TX_VAL_AGRI_ZS_UN,NGA_EA_PRD_AGRI_KD,NGA_TM_VAL_AGRI_ZS_UN,NGA_NV_AGR_TOTL_KD,NGA_NV_AGR_TOTL_KD_ZG,NGA_NV_AGR_TOTL_ZS,NGA_NV_AGR_TOTL_CN,NGA_NV_AGR_TOTL_KN,NGA_IS_AIR_PSGR,NGA_NV_AGR_TOTL_CD,NGA_IS_AIR_GOOD_MT_K1,NGA_EG_USE_COMM_CL_ZS,NGA_IS_AIR_DPRT,NGA_AG_LND_ARBL_ZS,NGA_AG_LND_ARBL_HA,NGA_AG_LND_ARBL_HA_PC,NGA_MS_MIL_TOTL_TF_ZS,NGA_MS_MIL_TOTL_P1,NGA_MS_MIL_MPRT_KD,NGA_FB_AST_NPER_ZS,NGA_TM_TAX_MRCH_BC_ZS,NGA_TM_TAX_MANF_BC_ZS,NGA_TM_TAX_TCOM_BC_ZS,NGA_SP_DYN_CBRT_IN,NGA_TM_TAX_TCOM_BR_ZS,CPI
0,0.704156,0.000000,0.151856,0.027786,0.083140,0.013892,0.446791,0.462126,0.055666,0.126347,0.955052,0.048776,1.0000,1.000000,0.009713,0.300201,0.305389,0.305389,0.284153,0.283174,0.281669,0.341708,0.593873,0.414020,0.328528,0.220654,0.670915,0.005810,0.021207,0.152846,0.390247,0.017081,0.233817,0.007978,0.249009,0.736432,0.193045,0.230490,0.460333,0.659131,...,0.374727,2.620565e-08,0.276570,0.395143,0.459073,0.346705,0.437740,0.391419,0.974030,0.299236,0.272700,0.707866,0.781774,0.781774,0.014325,0.029629,0.119455,0.019189,0.066668,0.347351,0.282937,0.019189,0.000000,0.322737,0.223494,0.273483,0.055394,0.823189,0.823189,0.832506,0.219620,0.217734,0.118585,0.14584,0.608864,0.957243,1.000000,0.957680,1.000000,6.900000
1,0.718776,0.007274,0.155562,0.041814,0.083066,0.024200,0.436029,0.469032,0.053862,0.123446,0.954918,0.047844,0.9925,0.964880,0.009758,0.302856,0.303384,0.303384,0.281129,0.280146,0.279861,0.332637,0.590652,0.416339,0.334111,0.210703,0.658240,0.005798,0.025381,0.151316,0.404506,0.021092,0.242581,0.014458,0.246571,0.730547,0.191828,0.226438,0.455892,0.618935,...,0.359421,2.441837e-08,0.273352,0.399493,0.472297,0.349890,0.453576,0.387052,0.973572,0.286124,0.260365,0.694378,0.781763,0.781763,0.013143,0.029885,0.118957,0.019565,0.066471,0.349438,0.265266,0.019565,0.032291,0.317065,0.212879,0.282259,0.063616,0.821399,0.821399,0.830720,0.245206,0.243563,0.108703,0.14584,0.610351,0.960806,0.972772,0.959887,0.966646,7.791667
2,0.733397,0.014547,0.159269,0.055842,0.082993,0.034508,0.425267,0.475938,0.052058,0.120545,0.954784,0.046911,0.9850,0.929760,0.009803,0.305510,0.301380,0.301380,0.278105,0.277119,0.278053,0.323566,0.587431,0.418659,0.339694,0.200751,0.645565,0.005787,0.029554,0.149787,0.418765,0.025104,0.251345,0.020938,0.244133,0.724662,0.190612,0.222386,0.451452,0.578739,...,0.344115,2.263108e-08,0.270133,0.403843,0.485521,0.353076,0.469412,0.382685,0.973114,0.273012,0.248030,0.680889,0.781753,0.781753,0.011961,0.030142,0.118460,0.019941,0.066275,0.351525,0.247594,0.019941,0.064582,0.311392,0.202264,0.291035,0.071838,0.819609,0.819609,0.828934,0.270793,0.269392,0.098821,0.14584,0.611839,0.964369,0.945543,0.962093,0.933292,8.683333
3,0.748018,0.021821,0.162976,0.069871,0.082920,0.044815,0.414504,0.482845,0.050253,0.117644,0.954649,0.045978,0.9775,0.894641,0.009848,0.308164,0.299375,0.299375,0.275080,0.274091,0.276244,0.314495,0.584210,0.420978,

In [32]:
n_steps = 3
df_test=test_data[-n_steps:]
CPI_test = df_latest['CPI'][-n_steps:]
print(CPI_test)
print(df_test.shape)
input_feature = input_data[:len(input_data)-n_steps] # to remove last 5 rows, follow the time step you set

238    27.666667
239    27.833333
240    28.000000
Name: CPI, dtype: float64
(3, 582)


In [18]:
input_feature.shape # check shape 

(238, 610)

In [0]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [34]:
# choose a number of time steps
from numpy import array
# convert into input/output
X, y = split_sequences(input_feature, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
# the array is (Total number of samples, time step, total number of dimensions)

(236, 3, 582) (236,)
[[0.70415577 0.         0.15185556 ... 1.         0.95768026 1.        ]
 [0.71877637 0.00727355 0.15556226 ... 0.97277166 0.95988668 0.96664623]
 [0.73339696 0.01454711 0.15926895 ... 0.94554332 0.96209309 0.93329247]] 8.683333333333334
[[0.71877637 0.00727355 0.15556226 ... 0.97277166 0.95988668 0.96664623]
 [0.73339696 0.01454711 0.15926895 ... 0.94554332 0.96209309 0.93329247]
 [0.74801756 0.02182066 0.16297565 ... 0.91831498 0.96429951 0.8999387 ]] 9.575000000000001
[[0.73339696 0.01454711 0.15926895 ... 0.94554332 0.96209309 0.93329247]
 [0.74801756 0.02182066 0.16297565 ... 0.91831498 0.96429951 0.8999387 ]
 [0.76263815 0.02909422 0.16668234 ... 0.89108664 0.96650592 0.86658493]] 10.466666666666667
[[0.74801756 0.02182066 0.16297565 ... 0.91831498 0.96429951 0.8999387 ]
 [0.76263815 0.02909422 0.16668234 ... 0.89108664 0.96650592 0.86658493]
 [0.77725875 0.03636777 0.17038904 ... 0.8638583  0.96871234 0.83323117]] 11.358333333333334
[[0.76263815 0.02909422 0

In [0]:
# final model
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 
# refer to this if necessary
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# model.save_weights('my_model_weights.h5')

In [36]:
# fit model
model.fit(X, y, epochs=200, verbose=0)#, validation_split=0.2, batch_size=5)

In [0]:
# demonstrate prediction
x_input = df_test
x_input = x_input.reshape((1, n_steps,X.shape[2] ))
yhat = model.predict(x_input, verbose=0)

Percentage Error

In [38]:
print(yhat)
actual = CPI_test[len(df_latest)-1]
print(float((abs(actual-yhat)/actual)*100))

[[28.0054]]
0.019284656271338463


In [39]:
weight_tensors = model.layers[1].weights
weight_names = list(map(lambda x: x.name, weight_tensors))
print(weight_names)

['dense_3/kernel:0', 'dense_3/bias:0']


In [40]:
for layer in model.layers:
        if "LSTM" in str(layer):
            weightLSTM = layer.get_weights()
warr,uarr, barr = weightLSTM
print(warr.shape) # warr is a numpy array of weights for inputs
print(uarr.shape) # uarr is a numpy array of weights for hidden units
print(barr.shape) # barr is a numpy array of bias

(582, 300)
(75, 300)
(300,)


In [41]:
print(warr)
print(uarr)
print(barr)

[[ 0.03749638 -0.02997526 -0.04632678 ... -0.00324335 -0.07669651
   0.0438001 ]
 [ 0.04962246 -0.06883258  0.02367057 ...  0.09105638 -0.023903
   0.01684958]
 [-0.03012713 -0.04754846 -0.01662874 ...  0.07281887 -0.03872166
  -0.02841193]
 ...
 [-0.06825925  0.04132728  0.07908301 ... -0.00705177 -0.08011845
   0.00313503]
 [-0.0440183  -0.0308653  -0.03427765 ... -0.08223053 -0.08163113
  -0.06002896]
 [ 0.00381582 -0.08496253  0.04720399 ... -0.03476831 -0.07972182
  -0.09328835]]
[[-0.07719569  0.07363452  0.0602853  ... -0.03115223 -0.08237154
   0.09580831]
 [ 0.05686579  0.02484708 -0.01817427 ... -0.03467558 -0.04065197
   0.02562957]
 [ 0.08081912 -0.02479234 -0.01371005 ...  0.04565225 -0.01285446
  -0.00993495]
 ...
 [ 0.01473714 -0.05161301 -0.03240162 ... -0.03766557 -0.04943531
   0.10727484]
 [ 0.05498635 -0.07392467 -0.02696898 ...  0.00527024 -0.0207024
   0.0723711 ]
 [ 0.02927645  0.09873219  0.07743395 ...  0.03289894 -0.0679174
  -0.02810632]]
[ 1.22587280e-02 -5.

In [0]:
# change df to percentage changes
# play with the rank/ totaln
# interpolate lesser 
# time step

In [0]:
time step = 5, CPI, indicator data
interpolate by year, 22 data points
# 20.441774368286133
interpolate by month, 262 DP
# 1.0591048002243042
interpolate by day, 7920 DP
# 0.015229816548526287 

After performing percentage change and normalizing 
interpolate by year time step 7
# 0.6501209735870361
# 351430.6875
interpolate by year time step 5
# 0.5464288592338562
# 320606.21875
interpolate by year time step 3
# 1.6997615098953247
interpolate by month time step 7
# 1.4450266361236572
interpolate by month time step 5
# 0.217756450176239


*** champion ***
interpolate by month time step 3
# 0.4332916736602783 25 units
# 0.15020249783992767 50 units 0.24396678805351257
# 0.08212704956531525 75 units
# 0.025785423815250397 75 units dense 15 , 1
# 0.09989678114652634 75 units dense 25 , 1
***  winner  # 0.016958503052592278 75 units dense 35 , 1 
*** end ***


binarize -1,0,1
year time step = 7
# 63.047935485839844
year time step = 5
# 59.16572189331055
year time step = 3
# 82.13848114013672
month time step = 7
# 11.621817588806152
month time step = 5
# 11.747268676757812
month time step = 3
# 15.268765449523926


binarize 0,1
year time step = 7
# 1.3483071327209473
year time step = 5
# 2.037656784057617
year time step = 3
# 4.179565906524658
month time step = 7
# 0.44504478573799133
month time step = 5
# 4.9152421951293945
month time step = 3
# 3.9173173904418945

binarize 1,2,3
year time step = 7
# 5.180117607116699
year time step = 5
# 4.566781520843506
year time step = 3
# 1.8392828702926636
month time step = 7
# 3.652085065841675
month time step = 5
# 4.432069778442383
month time step = 3
# 0.7650640606880188